# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [16]:
# Primeiro vamos precisar instalar o selenium
# O selenium roda as coisas em background
!pip install selenium

In [17]:
# Para que o selenium controle o seu navegador, ele precisa do webdriver
# chrome -> chromedriver
# firefox -> geckodriver
from selenium import webdriver # controla o navegador
from selenium.webdriver.common.keys import Keys # usar o teclado
from selenium.webdriver.common.by import By # localizar os itens no navegador

# criar o navegador
navegador = webdriver.Chrome()

# entraria no google e pesquisaria por cotação do dólar
navegador.get('https://www.google.com/')

# pegaria a cotação do dólar
'''navegador.find_element(qual parametro vc quer usar, qual o codigo do parametro)'''
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# entraria no google e pesquisaria a cotação do euro
navegador.get('https://www.google.com/')

# pegaria a cotação do euro

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# entraria no https://www.melhorcambio.com/ouro-hoje
navegador.get('https://www.melhorcambio.com/ouro-hoje')

# pegaria a cotação do ouro

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')

cotacao_ouro = cotacao_ouro.replace(',', '.')

print(cotacao_ouro)

navegador.quit()

5.012765
5.536899708
321.79


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [18]:
# importar a base de dados
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [19]:
# atualizar as cotações na base de dados
# cotação do dólar
'''tabela.loc[liha, coluna]'''
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
# cotação do euro
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
# cotação do ouro
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)
# atualizar o preço de compra e de venda na base de dados
# preço de compra = preço de original * cotacao
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']
# preço de venda = preço de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.012765,5012.714872,1.40,7017.800821
1,Carro Renault,4500.00,Euro,5.536900,24916.048686,2.00,49832.097372
2,Notebook Dell,899.99,Dólar,5.012765,4511.438372,1.70,7669.445233
3,IPhone,799.00,Dólar,5.012765,4005.199235,1.70,6808.838700
4,Carro Fiat,3000.00,Euro,5.536900,16610.699124,1.90,31560.328336
5,Celular Xiaomi,480.48,Dólar,5.012765,2408.533327,2.00,4817.066654
6,Joia 20g,20.00,Ouro,321.790000,6435.800000,1.15,7401.170000


### Agora vamos exportar a nova base de preços atualizada

In [20]:
# exportar essa base de dados para ter o resultado atualizado
tabela.to_excel('ProdutosNovo.xlsx', index=False)